In [28]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import os
import glob

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

In [2]:
def metrics_at_k(df, threshold=0.75, k=5, print_steps=False):

    cv_index = np.array([])
    job_index = np.array([])
    cv_label = np.array([])
    job_label = np.array([])
    similarity = np.array([])

    for c1 in set(df['job_label']):
        for c2 in set(df['job_label']):
            tmp_df = df[(df['cv_label'] == c1) & (df['job_label'] == c2)].sort_values(by='similarity', ascending=False).head(k)
            cv_index = np.append(cv_index, tmp_df['cv_index'])
            job_index = np.append(job_index, tmp_df['job_index'])
            cv_label = np.append(cv_label, tmp_df['cv_label'])
            job_label = np.append(job_label, tmp_df['job_label'])
            similarity = np.append(similarity, tmp_df['similarity'])

    new_df = pd.DataFrame({
        'cv_index': cv_index,
        'job_index': job_index,
        'cv_label': cv_label,
        'job_label': job_label,
        'similarity': similarity
    })

    true_positive = np.array([])
    true_negative = np.array([])
    false_positive = np.array([])
    false_negative = np.array([])

    for i in range(len(new_df)):
        if new_df.iloc[i]["cv_label"] == new_df.iloc[i]["job_label"]:
            if new_df.iloc[i]["similarity"] > threshold:
                true_positive = np.append(true_positive, new_df.iloc[i]["similarity"])
            else:
                false_negative = np.append(false_negative, new_df.iloc[i]["similarity"])
        else:
            if new_df.iloc[i]["similarity"] > threshold:
                false_positive = np.append(false_positive, new_df.iloc[i]["similarity"])
            else:
                true_negative = np.append(true_negative, new_df.iloc[i]["similarity"])  
    
    accuracy_k = (len(true_positive) + len(true_negative)) / (
        len(true_positive)
        + len(true_negative)
        + len(false_positive)
        + len(false_negative)
    )

    try:
        recall_k = len(true_positive) / (len(true_positive) + len(false_negative))
        precision_k = len(true_positive) / (len(true_positive) + len(false_positive))
        f1_score_k = 2 * recall_k * precision_k / (recall_k + precision_k)
    except Exception as e:
        recall_k = 0
        precision_k = 0
        f1_score_k = 0
    
    if print_steps:
        print(f"TP: {len(true_positive)}")  
        print(f"TN: {len(true_negative)}")
        print(f"FP: {len(false_positive)}")
        print(f"FN: {len(false_negative)}")

    return accuracy_k, f1_score_k, recall_k, precision_k

In [3]:
def accuracy_f1_score(df, threshold=0.75):
    true_positive = np.array([])
    true_negative = np.array([])

    false_positive = np.array([])
    false_negative = np.array([])

    for i in range(len(df)):
        if df.iloc[i]["cv_label"] == df.iloc[i]["job_label"]:
            if df.iloc[i]["similarity"] > threshold:
                true_positive = np.append(true_positive, df.iloc[i]["similarity"])
            else:
                false_negative = np.append(false_negative, df.iloc[i]["similarity"])
        else:
            if df.iloc[i]["similarity"] > threshold:
                false_positive = np.append(false_positive, df.iloc[i]["similarity"])
            else:
                true_negative = np.append(true_negative, df.iloc[i]["similarity"])

    # accuracy with threshold
    accuracy = (len(true_positive) + len(true_negative)) / (
        len(true_positive)
        + len(true_negative)
        + len(false_positive)
        + len(false_negative)
    )

    # accuracy based on mean
    # count = (true_positive.mean() + true_negative.mean()) \
    #         / (true_positive.mean() + true_negative.mean() + false_positive.mean() + false_negative.mean())
    try:
        recall = len(true_positive) / (len(true_positive) + len(false_negative))
        precision = len(true_positive) / (len(true_positive) + len(false_positive))
        f1_score = 2 * recall * precision / (recall + precision)
    except Exception as e:
        recall = 0
        precision = 0
        f1_score = 0

    # compute recall@5 & recall@10
    accuracy_5, f1_score_5, recall_5, precision_5 = metrics_at_k(df, threshold=threshold, k=5)
    accuracy_10, f1_score_10, recall_10, precision_10 = metrics_at_k(df, threshold=threshold, k=10)
    accuracy_100, f1_score_100, recall_100, precision_100 = metrics_at_k(df, threshold=threshold, k=100)

    return accuracy, f1_score, recall, precision, accuracy_5, f1_score_5, recall_5, precision_5, accuracy_10, f1_score_10, recall_10, precision_10, accuracy_100, f1_score_100, recall_100, precision_100

In [4]:
def accuracy_evaluation(
    file_values: [(str, str, str)], method_str, th_start=0.5, th_end=0.95, th_step=0.05
):
    threshold_settings = np.arange(th_start, th_end, th_step)

    model_arr = []
    level_arr = []
    threshold_arr = []
    accuracy_arr = []
    f1_score_arr = []

    recall_arr = []
    precision_arr = []

    accuracy_arr_5 = []
    f1_score_arr_5 = []
    recall_arr_5 = []
    precision_arr_5 = []

    accuracy_arr_10 = []
    f1_score_arr_10 = []
    recall_arr_10 = []
    precision_arr_10 = []

    accuracy_arr_100 = []
    f1_score_arr_100 = []
    recall_arr_100 = []
    precision_arr_100 = []

    with tqdm(total=len(file_values) * len(threshold_settings)) as pbar:
        for file, model, level in file_values:
            df = pd.read_csv(file)
            for threshold in threshold_settings:
                # accuracy, f1_score, recall, precision = accuracy_f1_score(df, threshold)
                accuracy, f1_score, recall, precision, accuracy_5, f1_score_5, recall_5, precision_5, accuracy_10, f1_score_10, recall_10, precision_10, accuracy_100, f1_score_100, recall_100, precision_100 = accuracy_f1_score(
                    df, threshold
                )
                model_arr.append(model)
                level_arr.append(level)
                threshold_arr.append(threshold)
                accuracy_arr.append(accuracy)
                f1_score_arr.append(f1_score)
                recall_arr.append(recall)
                precision_arr.append(precision)
                
                accuracy_arr_5.append(accuracy_5)
                f1_score_arr_5.append(f1_score_5)
                recall_arr_5.append(recall_5)
                precision_arr_5.append(precision_5)

                accuracy_arr_10.append(accuracy_10)
                f1_score_arr_10.append(f1_score_10)
                recall_arr_10.append(recall_10)
                precision_arr_10.append(precision_10)

                accuracy_arr_100.append(accuracy_100)
                f1_score_arr_100.append(f1_score_100)
                recall_arr_100.append(recall_100)
                precision_arr_100.append(precision_100)

                pbar.update(1)

    df_analysis = pd.DataFrame(
        {
            "model": model_arr,
            "level": level_arr,
            "threshold": threshold_arr,
            "method": method_str,
            "accuracy": accuracy_arr,
            "f1_score": f1_score_arr,
            "recall": recall_arr,
            "precision": precision_arr,
            "accuracy@5": accuracy_arr_5,
            "f1_score@5": f1_score_arr_5,
            "recall@5": recall_arr_5,
            "precision@5": precision_arr_5,
            "accuracy@10": accuracy_arr_10,
            "f1_score@10": f1_score_arr_10,
            "recall@10": recall_arr_10,
            "precision@10": precision_arr_10,
            "accuracy@100": accuracy_arr_100,
            "f1_score@100": f1_score_arr_100,
            "recall@100": recall_arr_100,
            "precision@100": precision_arr_100,
        }
    )
    return df_analysis

In [5]:
def get_files_and_values(file):
    file_values = []
    level = ''
    model = ''

    tmp_arr = os.path.basename(file).split('_')
    model = '_'.join(tmp_arr[:2]).split('.')[0]

    if len(tmp_arr) > 2:
        level = '_'.join(tmp_arr[2:]).split('.')[0]
    else:
        level = ''

    file_values.append((file, model, level))

    return file_values

In [35]:
def generate_confusion_matrix_at_k(df, k, threshold, save_path):
    labels = list(set(df['cv_label'].to_list()))

    cv_index = np.array([])
    job_index = np.array([])
    cv_label = np.array([])
    job_label = np.array([])
    similarity = np.array([])

    for c1 in set(df['job_label']):
        for c2 in set(df['job_label']):
            tmp_df = df[(df['cv_label'] == c1) & (df['job_label'] == c2)].sort_values(by='similarity', ascending=False).head(k)
            cv_index = np.append(cv_index, tmp_df['cv_index'])
            job_index = np.append(job_index, tmp_df['job_index'])
            cv_label = np.append(cv_label, tmp_df['cv_label'])
            job_label = np.append(job_label, tmp_df['job_label'])
            similarity = np.append(similarity, tmp_df['similarity'])

    new_df = pd.DataFrame({
        'cv_index': cv_index,
        'job_index': job_index,
        'cv_label': cv_label,
        'job_label': job_label,
        'similarity': similarity
    })

    filtered_data = new_df[new_df["similarity"] > threshold]
    confusion = confusion_matrix(
                    filtered_data['cv_label'], filtered_data['job_label']
    )

    plt.figure(figsize=(8, 7))
    sns.heatmap(
        confusion,
        annot=True,
        fmt="d",
        cmap="Blues",
        xticklabels=labels,
        yticklabels=labels,
    )
    plt.xlabel("Job Announces")
    plt.ylabel("Resumes")
    plt.tight_layout()

    plt.savefig(save_path, dpi=300)
    plt.close()

In [30]:
# df = pd.read_csv("./data/result_df/new_data_method2/base_model.csv", index_col=0)
df = get_files_and_values("./data/result_df/new_data_method2/base_model.csv")
# df = get_files_and_values("./data/result_df/new_data_method2/concept_model_alt.csv")

In [10]:
tmp_df = accuracy_evaluation(df, "method2", th_start=0.5, th_end=0.95, th_step=0.05)
tmp_df

  0%|          | 0/9 [00:00<?, ?it/s]

,model,level,threshold,method,accuracy,f1_score,recall,precision,accuracy@5,f1_score@5,recall@5,precision@5,accuracy@10,f1_score@10,recall@10,precision@10,accuracy@100,f1_score@100,recall@100,precision@100
0,base_model,,0.50,method2,0.100201,0.181851,1.000000,0.100020,0.100,0.181818,1.00,0.100000,0.100,0.181818,1.00,0.100000,0.1000,0.181818,1.000,0.100000
1,base_model,,0.55,method2,0.105327,0.182252,0.996985,0.100293,0.100,0.181818,1.00,0.100000,0.100,0.181818,1.00,0.100000,0.1000,0.181818,1.000,0.100000
2,base_model,,0.60,method2,0.304573,0.217738,0.967839,0.122667,0.100,0.181818,1.00,0.100000,0.100,0.181818,1.00,0.100000,0.1081,0.183167,1.000,0.100817
3,base_model,,0.65,method2,0.678844,0.325488,0.774874,0.206012,0.104,0.182482,1.00,0.100402,0.122,0.185529,1.00,0.102249,0.4587,0.269796,1.000,0.155933
4,base_model,,0.70,method2,0.851859,0.376217,0.446734,0.324927,0.324,0.228311,1.00,0.128866,0.383,0.244798,1.00,0.139470,0.7942,0.433993,0.789,0.299317
5,base_model,,0.75,method2,0.891307,0.284486,0.216080,0.416263,0.606,0.336700,1.00,0.202429,0.660,0.353612,0.93,0.218310,0.8827,0.423020,0.430,0.416263
6,base_model,,0.80,method2,0.896533,0.144578,0.087437,0.417266,0.738,0.384977,0.82,0.251534,0.771,0.369146,0.67,0.254753,0.8931,0.245589,0.174,0.417266
7,base_model,,0.85,method2,0.898543,0.069156,0.037688,0.418994,0.822,0.377622,0.54,0.290323,0.857,0.401674,0.48,0.345324,0.8971,0.127226,0.075,0.418994
8,base_model,,0.90,method2,0.898392,0.025072,0.013065,0.309524,0.848,0.321429,0.36,0.290323,0.871,0.262857,0.23,0.306667,0.8968,0.047970,0.026,0.309524


In [37]:
# generate confusion matrices
df_conf = pd.read_csv(df[0][0], index_col=0)
generate_confusion_matrix_at_k(df_conf, 5, 0.8, "./data/recall@K/base_model/recall@5.png")
generate_confusion_matrix_at_k(df_conf, 10, 0.8, "./data/recall@K/base_model/recall@10.png")
generate_confusion_matrix_at_k(df_conf, 100, 0.7, "./data/recall@K/base_model/recall@100.png")

In [102]:
model_name = '\_'.join(tmp_df['model'][0].split('_'))
k = 5
print(f"{model_name} & {tmp_df['level'][0]} & {k} & {tmp_df[f'accuracy@{k}'][0]:.4f} & {tmp_df[f'recall@{k}'][0]:.4f} & {tmp_df[f'precision@{k}'][0]:.4f}& {tmp_df[f'f1_score@{k}'][0]:.4f} & {tmp_df['threshold'][0]:.2f}\\\\")
k = 10
print(f"{model_name} & {tmp_df['level'][0]} & {k} & {tmp_df[f'accuracy@{k}'][0]:.4f} & {tmp_df[f'recall@{k}'][0]:.4f} & {tmp_df[f'precision@{k}'][0]:.4f}& {tmp_df[f'f1_score@{k}'][0]:.4f} & {tmp_df['threshold'][0]:.2f}\\\\")
k = 100
print(f"{model_name} & {tmp_df['level'][0]} & {k} & {tmp_df[f'accuracy@{k}'][0]:.4f} & {tmp_df[f'recall@{k}'][0]:.4f} & {tmp_df[f'precision@{k}'][0]:.4f}& {tmp_df[f'f1_score@{k}'][0]:.4f} & {tmp_df['threshold'][0]:.2f}\\\\")

concept\_model & alt & 5 & 0.3140 & 1.0000 & 0.1272& 0.2257 & 0.70\\
concept\_model & alt & 10 & 0.3750 & 1.0000 & 0.1379& 0.2424 & 0.70\\
concept\_model & alt & 100 & 0.7861 & 0.8030 & 0.2925& 0.4288 & 0.70\\


In [110]:
df = get_files_and_values("./data/result_df/new_data_method2/concept_model_alt.csv")
tmp_df = accuracy_evaluation(df, "method2", th_start=0.5, th_end=0.95, th_step=0.05)
tmp_df

  0%|          | 0/9 [00:00<?, ?it/s]

,model,level,threshold,method,accuracy,f1_score,recall,precision,accuracy@5,f1_score@5,recall@5,precision@5,accuracy@10,f1_score@10,recall@10,precision@10,accuracy@100,f1_score@100,recall@100,precision@100
0,concept_model,alt,0.50,method2,0.100201,0.181851,1.000000,0.100020,0.100,0.181818,1.00,0.100000,0.100,0.181818,1.00,0.100000,0.1000,0.181818,1.000,0.100000
1,concept_model,alt,0.55,method2,0.104121,0.182052,0.996985,0.100172,0.100,0.181818,1.00,0.100000,0.100,0.181818,1.00,0.100000,0.1000,0.181818,1.000,0.100000
2,concept_model,alt,0.60,method2,0.290754,0.214842,0.970352,0.120793,0.100,0.181818,1.00,0.100000,0.100,0.181818,1.00,0.100000,0.1063,0.182866,1.000,0.100634
3,concept_model,alt,0.65,method2,0.665427,0.320335,0.788442,0.200999,0.100,0.181818,1.00,0.100000,0.116,0.184502,1.00,0.101626,0.4380,0.262467,1.000,0.151057
4,concept_model,alt,0.70,method2,0.848492,0.378222,0.460804,0.320742,0.314,0.225734,1.00,0.127226,0.375,0.242424,1.00,0.137931,0.7861,0.428838,0.803,0.292532
5,concept_model,alt,0.75,method2,0.890704,0.290375,0.223618,0.413953,0.596,0.331126,1.00,0.198413,0.651,0.352505,0.95,0.216401,0.8815,0.428916,0.445,0.413953
6,concept_model,alt,0.80,method2,0.895930,0.147386,0.089950,0.407745,0.726,0.374429,0.82,0.242604,0.761,0.359249,0.67,0.245421,0.8919,0.248784,0.179,0.407745
7,concept_model,alt,0.85,method2,0.898543,0.070870,0.038693,0.420765,0.820,0.375000,0.54,0.287234,0.856,0.400000,0.48,0.342857,0.8971,0.130178,0.077,0.420765
8,concept_model,alt,0.90,method2,0.898392,0.025072,0.013065,0.309524,0.848,0.321429,0.36,0.290323,0.871,0.262857,0.23,0.306667,0.8968,0.047970,0.026,0.309524


In [111]:
model_name = '\_'.join(tmp_df['model'][0].split('_'))
INDEX = 7
k = 5
print(f"{model_name} & {tmp_df['level'][INDEX]} & {k} & {tmp_df[f'accuracy@{k}'][INDEX]:.4f} & {tmp_df[f'recall@{k}'][INDEX]:.4f} & {tmp_df[f'precision@{k}'][INDEX]:.4f}& {tmp_df[f'f1_score@{k}'][INDEX]:.4f} & {tmp_df['threshold'][INDEX]:.2f}\\\\")
INDEX = 7
k = 10
print(f"{model_name} & {tmp_df['level'][INDEX]} & {k} & {tmp_df[f'accuracy@{k}'][INDEX]:.4f} & {tmp_df[f'recall@{k}'][INDEX]:.4f} & {tmp_df[f'precision@{k}'][INDEX]:.4f}& {tmp_df[f'f1_score@{k}'][INDEX]:.4f} & {tmp_df['threshold'][INDEX]:.2f}\\\\")
INDEX = 5
k = 100
print(f"{model_name} & {tmp_df['level'][INDEX]} & {k} & {tmp_df[f'accuracy@{k}'][INDEX]:.4f} & {tmp_df[f'recall@{k}'][INDEX]:.4f} & {tmp_df[f'precision@{k}'][INDEX]:.4f}& {tmp_df[f'f1_score@{k}'][INDEX]:.4f} & {tmp_df['threshold'][INDEX]:.2f}\\\\")

concept\_model & alt & 5 & 0.8200 & 0.5400 & 0.2872& 0.3750 & 0.85\\
concept\_model & alt & 10 & 0.8560 & 0.4800 & 0.3429& 0.4000 & 0.85\\
concept\_model & alt & 100 & 0.8815 & 0.4450 & 0.4140& 0.4289 & 0.75\\


In [8]:
df_tmp = pd.read_csv("./data/result_df/new_data_method2/base_model.csv", index_col=0)
accuracy, f1_score, recall, precision = metrics_at_k(df_tmp, threshold=0.7, k=10, print_steps=True)

# recall = TP / (TP + FN))
# precision = TP / (TP + FP)
# f1_score = 2 * recall * precision / (recall + precision)

print(f"accuracy: {accuracy}, f1_score: {f1_score}, recall: {recall}, precision: {precision}")

TP: 100
TN: 283
FP: 617
FN: 0
accuracy: 0.383, f1_score: 0.2447980416156671, recall: 1.0, precision: 0.1394700139470014
